In [95]:
import os
import openai
import faiss
import pdfminer
from pdfminer.high_level import extract_text
from langchain.embeddings import OpenAIEmbeddings 
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS

# Outline
1. Extract text from all pdf files and store list of strings
2. Merge all strings in the documents list and then create chunks
3. 

In [2]:
os.environ['OPENAI_API_KEY'] = 'enter_api_key'

In [33]:
folder_path = "/Users/mamtaudai/Downloads"


In [34]:
documents = []

In [54]:
files = os.listdir(folder_path)
# gets all filenames inside folder_path.
print(files)

['.DS_Store', '.localized', 'MamtaUdai_IITK_CV.pdf', 'AI Engineer - Asociate - Intern.docx', 'SQL-Assignment (1).pdf', 'photo2.jpeg', 'Assistant Data Scientist.docx', 'Senior Data Scientist.docx', 'sample.txt', 'profile_pic.jpeg']


In [101]:
for file in files:
    if file.endswith('.pdf'):
        file_path = os.path.join(folder_path, file)
        print(f"Processing file: {file_path}")  
        text = extract_text(file_path)
        documents.append(text)

Processing file: /Users/mamtaudai/Downloads/MamtaUdai_IITK_CV.pdf
Processing file: /Users/mamtaudai/Downloads/SQL-Assignment (1).pdf


In [102]:
type(documents)

list

In [103]:
len(documents)

11

In [104]:
type(documents[0])

str

In [55]:
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
# define each chunk will have 500 characters
print(splitter)


In [56]:
chunks = splitter.split_text("\n\n".join(documents))
# Combines all extracted texts into one string and splits it.
print(chunks)

['MAMTA UDAI \n\n+91-9782991795  \nmamtaudai98@gmail.com \n\nEDUCATION \n\n▪  IIT Kanpur, Bachelor of Science (BS) in Mathematics and Scientific Computing, 2023   \n\nEXPERIENCE \n\nFreelance Math AI Trainer    \nOutlier.ai| Oct 2024 - Present', '▪  Developed and tested over 500 high-quality prompts for AI models, focusing on math-related queries (geometry, \n\nalgebra, and statistics), achieving 95% alignment with quality benchmarks. \n\n▪  Reviewed and evaluated AI model to ensure accuracy, consistency, and quality in handling math-related queries. \n▪  Designed and implemented a structured evaluation process, improving assessment efficiency and reliability by 20%.', 'PROJECTS \nCredit Card Fraud Detection                                                                                                                             \n▪  Executed comprehensive data pre-processing including Exploratory Data Analysis (EDA), Feature \n\nEngineering and Anomaly Detection to prepare data for t

In [74]:
type(chunks)

list

In [75]:
type(chunks[0])

str

In [76]:
len(chunks)

20

In [53]:
persist_path = 'faiss_index'
# defines the directory name where the FAISS index will be stored on disk

In [10]:
embedding_model = OpenAIEmbeddings(model='text-embedding-3-small')
# Converts text chunks into vector representations.


/var/folders/kd/1rh3tcw579x8kh9plm897wfh0000gp/T/ipykernel_43778/621455296.py:1: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings(model='text-embedding-3-small')


In [57]:
# if os.path.exists(persist_path):
#     print("Loading existing FAISS index...")
#     os.system(f"rm -rf {persist_path}")

vectorstore = FAISS.from_texts(chunks, embedding_model)
vectorstore.save_local(persist_path) 

#text chunks into embedding using embedding_model and then storing embedding into FAISS VectorDtabase

Loading existing FAISS index...


In [84]:
query = "What is mamta's email id"
retriever = vectorstore.as_retriever(search_kwargs={'k':3})
# Retrieves the top 3 most similar text chunks to the query.
relevant_docs = retriever.get_relevant_documents(query)
# Retrieves relevant chunks.



In [106]:
relevant_docs[0].page_content

'MAMTA UDAI \n\n+91-9782991795  \nmamtaudai98@gmail.com \n\nEDUCATION \n\n▪  IIT Kanpur, Bachelor of Science (BS) in Mathematics and Scientific Computing, 2023   \n\nEXPERIENCE \n\nFreelance Math AI Trainer    \nOutlier.ai| Oct 2024 - Present'

In [90]:
if not relevant_docs:
    response = "No relevant information found. Try rephrasing your question."
else:
    context = "\n\n".join(doc.page_content for doc in relevant_docs)
    llm = ChatOpenAI(model="gpt-4o-mini")
    prompt = (
        f"Using the following context, answer the question concisely:\n\n"
        f"Context:\n{context}\n\n"
        f"Question: {query}\n\n"
        f"Answer:"
    )
    response = llm.invoke(prompt).content
    final_response = ''
    if response == None or response == '':
        final_response = "No response generated."
    else:    
        final_response = response

In [107]:
type(context)

str

In [96]:
print(final_response)

mamtaudai98@gmail.com


In [92]:
print(prompt)

Using the following context, answer the question concisely:

Context:
MAMTA UDAI 

+91-9782991795  
mamtaudai98@gmail.com 

EDUCATION 

▪  IIT Kanpur, Bachelor of Science (BS) in Mathematics and Scientific Computing, 2023   

EXPERIENCE 

Freelance Math AI Trainer    
Outlier.ai| Oct 2024 - Present

LANGUAGES AND SKILLS 

▪  C, Python, MATLAB, MySQL, NoSQL 
▪  MS-Excel, Power BI, Tableau 

CERTIFICATIONS   

▪  Machine Learning Specialization from Coursera 
▪  Data Science Bootcamp from UpGrad 

 
 
 
 
 
 


MAMTA UDAI 

+91-9782991795  
mamtaudai98@gmail.com 

EDUCATION 

§  IIT Kanpur, Bachelor of Science (BS) in Mathematics and Scientific Computing, 2023   

EXPERIENCE

LANGUAGES AND SKILLS 

§  Programming Languages: Python, C, R, SQL, Matlab 
§  Tools and frameworks: Pandas, Keras, Tensorflow, Scikit-learn, LLM, GenAI, Huggingface, MS-Excel, Power BI, 

Tableau, OpenAI, Elasticsearch 

CERTIFICATIONS   

§  Machine Learning Specialization from Coursera 
§  Data Science Bootcamp 